In [1]:
# conda environment (project)
# /home/student/.conda/envs/project/bin/python 
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os, gc
import torch

from transformers import set_seed
from datasets import load_dataset
from evaluate import *
from arc.arc_new import ARCSolver

from datasets import Dataset
from utils import render_grid

In [2]:
# prepare the test dataset
data_path = "/workspace/dataset"
dataset, task_list = load_data(data_path)
df300 = sample_data(dataset, task_list, n_row=30000) 
df300.head(5) 

Imported 300 different tasks in the dataset.
# of samples in a task: min(82), Q1(1000), Q2(1000), Q3(1000), max(1000), mean(897.7)


,task,train,test_input,test_output,test
0,6d58a25d,"[{'input': [[2, 7, 7, 7, 7, 0, 0], [2, 7, 7, 7...","[{'input': [[5, 5, 5, 5, 5, 5, 2], [5, 5, 5, 5...","[[[5, 5, 5, 8, 5, 5, 2], [5, 5, 5, 8, 5, 5, 2]...","[{'input': [[5, 5, 5, 5, 5, 5, 2], [5, 5, 5, 5..."
1,b0c4d837,"[{'input': [[4, 4, 4, 4, 4, 4, 4, 4, 2, 5], [4...","[{'input': [[1, 1, 4, 4, 4, 4, 4], [9, 1, 4, 4...","[[[1, 1, 2, 3, 9, 1, 2], [9, 1, 2, 3, 9, 1, 2]...","[{'input': [[1, 1, 4, 4, 4, 4, 4], [9, 1, 4, 4..."
2,746b3537,"[{'input': [[9, 2], [3, 4], [4, 3], [3, 3], [4...","[{'input': [[3], [3], [7], [7], [8], [3], [7],...","[[[3, 7, 8], [3, 7, 0], [3, 7, 0], [3, 0, 0]]]","[{'input': [[3], [3], [7], [7], [8], [3], [7],..."
3,25d8a9c8,"[{'input': [[7, 7, 7, 7, 7, 7, 7, 7, 7, 7], [7...","[{'input': [[2, 3, 3, 3, 3, 2, 3, 3, 3], [3, 2...","[[[2, 7, 7, 7, 7, 2], [2, 2, 3, 3, 3, 3], [3, ...","[{'input': [[2, 3, 3, 3, 3, 2, 3, 3, 3], [3, 2..."
4,25ff71a9,"[{'input': [[8, 2, 9, 9, 9], [9, 9, 9, 9, 9], ...","[{'input': [[6, 6, 6, 6, 6, 1, 6, 6, 6, 6], [6...","[[[2, 0, 6, 4, 6, 1, 6, 6, 6, 6], [7, 1, 6, 1,...","[{'input': [[6, 6, 6, 6, 6, 1, 6, 6, 6, 6], [6..."


In [4]:
# prepare samples for each task
task_samples = []
for t in range(300):
    df = sample_data(dataset, task_list, n_row=1000, indices=[t])
    task_samples.append(df)

In [6]:
# Visualize a task (EDA)
task_idx = 11 # select which task you want to examine
n_sample = 1
for task_idx in hard_tasks[:50]:
    print(task_idx)
    for data in Dataset.from_pandas(task_samples[task_idx]).shuffle().select(range(n_sample)):
        for case in data['train']:
            print("==================================================")
            print("Example input")
            render_grid(case['input'])
            print("Example output")
            render_grid(case['output'])
            break
        print("==================================================")
        print("Example test input")
        render_grid(data['test'][0]['input'])
        print("Example test output")
        render_grid(data['test'][0]['output'])
    print("==================================================")

5
Example input


                [4, 4, 4, 4, 4, 4, 4]
                [4, 0, 4, 4, 4, 4, 4]
                [4, 0, 4, 4, 4, 4, 4]
                [4, 4, 0, 0, 4, 4, 4]
                [4, 4, 4, 0, 0, 4, 4]
                [4, 4, 0, 4, 4, 4, 4]
                [4, 4, 4, 4, 4, 4, 4]

Example output


                  [0, 0, 4, 4, 4, 4, 4, 4]
                  [0, 0, 4, 4, 4, 4, 4, 4]
                  [0, 0, 4, 4, 4, 4, 4, 4]
                  [0, 0, 4, 4, 4, 4, 4, 4]
                  [4, 4, 0, 0, 0, 0, 4, 4]
                  [4, 4, 0, 0, 0, 0, 4, 4]
                  [4, 4, 4, 4, 0, 0, 0, 0]
                  [4, 4, 4, 4, 0, 0, 0, 0]
                  [4, 4, 0, 0, 4, 4, 4, 4]
                  [4, 4, 0, 0, 4, 4, 4, 4]

Example test input


...

                      [8, 8, 8, 8, 8, 8, 1, 8, 0, 3]
                      [8, 5, 8, 8, 5, 8, 4, 3, 1, 2]
                      [4, 4, 7, 8, 5, 8, 8, 8, 8, 2]

Example test input


              [7, 4, 4, 3, 4, 4]
              [4, 4, 6, 7, 5, 4]
              [7, 4, 4, 3, 4, 4]
              [4, 4, 6, 7, 5, 4]

Example test output


              [7, 4, 4, 3, 4, 4]
              [4, 4, 6, 7, 5, 4]

In [5]:
simple_tasks = []
hard_tasks = []
for task_idx in range(300):
    check = True
    for data in Dataset.from_pandas(task_samples[task_idx]).shuffle().select(range(3)):
        for case in data['train']:
            wi, hi = len(case['input'][0]), len(case['input'])
            wo, ho = len(case['output'][0]), len(case['output'])
            if (wi!=wo) or (hi!=ho): check = False
        case = data['test'][0]
        wi, hi = len(case['input'][0]), len(case['input'])
        wo, ho = len(case['output'][0]), len(case['output'])
        if (wi!=wo) or (hi!=ho): check = False
    if check: simple_tasks.append(task_idx)
    else: hard_tasks.append(task_idx)
print(simple_tasks)

[0, 1, 2, 3, 4, 7, 9, 10, 11, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 38, 39, 40, 42, 43, 45, 46, 47, 48, 50, 52, 54, 59, 60, 61, 63, 64, 65, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 85, 86, 87, 88, 89, 90, 92, 94, 96, 97, 99, 100, 102, 105, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 119, 121, 124, 125, 128, 129, 131, 132, 133, 136, 137, 139, 140, 141, 142, 145, 146, 147, 148, 149, 150, 152, 153, 154, 158, 162, 165, 166, 170, 173, 175, 176, 177, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 217, 218, 220, 221, 222, 224, 225, 227, 228, 229, 230, 231, 233, 234, 236, 237, 239, 240, 241, 242, 243, 245, 246, 247, 249, 252, 255, 257, 261, 262, 263, 265, 269, 271, 275, 278, 279, 283, 284, 285, 287, 289, 294, 295, 296, 298]


In [6]:
set_seed(1234567890)
token = os.environ.get("HF_TOKEN", None)
solver = ARCSolver(model_id="Qwen/Qwen3-4B", hf_token=token)

In [9]:
# solver.prepare_train()
n_train = len(hard_tasks)*1000
n_eval = 500
dfsimple = sample_data(dataset, task_list, n_row=n_train+n_eval, indices=simple_tasks, random=56)
dfhard = sample_data(dataset, task_list, n_row=n_train+n_eval, indices=hard_tasks, random=56)
train_dataset = Dataset.from_pandas(dfsimple).select(range(n_train))
# solver.train(train_dataset)

In [7]:
n_eval = 100
solver.prepare_evaluation(select_adapter="20250604_112335") # make sure you set the right model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
# evaluate our model (eval set)
scores = []
n_eval = 10
# scores_task = []
# for task in [15]: #range(20):
eval_dataset = Dataset.from_pandas(dfhard).select(range(n_train, n_train+n_eval)) # Dataset.from_pandas(task_samples[task]).select(range(n_eval))
for eval_data in tqdm(eval_dataset):
    print("============================================")
    print("Test input")
    render_grid(eval_data["test"][0]['input'])

    print("Predict output")
    preds = solver.predict(eval_data)
    if preds is not None: render_grid(preds)

    print("Test output")
    render_grid(eval_data["test"][0]['output'])
    print("============================================")
    if preds is None: s = 0
    else: s = check_match(preds, eval_data["test"][0]["output"])
    scores.append(s)
score = np.array(scores).mean() * 100
# scores_task.append(score)
print(f"Evaluation score: {score:.2f}", flush=True)
scores = []

# from matplotlib import pyplot as plt
# x = np.arange(20)
# plt.bar(x, scores_task)
# plt.xticks(x, list(range(20)))
# plt.show()

  0%|          | 0/10 [00:00<?, ?it/s]

Test input


          [2, 6, 6, 2]
          [2, 2, 7, 2]

Predict output
<|im_start|>system
You are an expert for solving logical puzzles. You are given some pairs of input and output as 2D grid with the same underlying spatial pattern between them. You have to logically infer the exact rule that transforms each input to the corresponding output.
<|im_end|>
<|im_start|>user
Here are the example input and output pairs from which you should learn the underlying rule to later predict the output for the given test input:
----------------------------------------
input:
6
6
6
0
6
6
6
6
output:
8888880
6666666

input:
929927
997992
output:
8888877
8888222
9999999

input:
73
33
73
47
output:
8884
8777
3333


----------------------------------------
Now, solve the following puzzle based on its input grid by applying the rules you have learned from the training data.:
----------------------------------------
input:
66
59
69
66

----------------------------------------
What is the output grid? Only provide the output grid in the form as in the example i

        [7 6 0]
        [7 6 0]
        [7 6 0]
        [7 0 0]

...

Predict output
<|im_start|>system
You are an expert for solving logical puzzles. You are given some pairs of input and output as 2D grid with the same underlying spatial pattern between them. You have to logically infer the exact rule that transforms each input to the corresponding output.
<|im_end|>
<|im_start|>user
Here are the example input and output pairs from which you should learn the underlying rule to later predict the output for the given test input:
----------------------------------------
input:
22222
23333
23444
23333
output:
2222222222
2222222222
2203333333
2230333333
2233444444
2233444444
2230333333
2203333333

input:
371
371
377
377
333
output:
337711
337711
337711
337711
337077
330777
337777
337777
333333
333333

input:
2171
2171
2171
2111
2222
output:
22117711
22117711
22117711
22117711
22117711
22117711
22101101
22011110
22222222
22222222


----------------------------------------
Now, solve the following puzzle based on its input grid by applying the rules you have 

                      [9 9 9 9 9 9 9 9 9 9]
                      [9 3 3 3 3 3 3 3 9 9]
                      [9 3 3 3 3 3 3 3 9 9]
                      [9 3 3 3 3 3 3 3 9 9]
                      [9 3 3 3 3 3 3 3 9 9]
                      [9 3 3 3 3 3 3 3 9 9]
                      [9 3 3 8 3 3 3 3 9 9]
                      [9 3 3 3 3 3 3 3 9 9]
                      [9 9 9 9 9 9 9 9 9 9]
                      [9 9 9 9 9 9 9 9 9 9]

Test output


                  [9, 9, 9, 9, 9, 9, 9, 9]
                  [9, 9, 9, 9, 9, 9, 9, 9]
                  [3, 3, 3, 3, 3, 2, 9, 9]
                  [3, 3, 3, 3, 2, 3, 9, 9]
                  [3, 3, 3, 2, 3, 3, 9, 9]
                  [3, 3, 2, 3, 3, 3, 9, 9]
                  [8, 8, 3, 3, 3, 3, 9, 9]
                  [8, 8, 3, 3, 3, 3, 9, 9]
                  [3, 3, 2, 3, 3, 3, 9, 9]
                  [3, 3, 3, 2, 3, 3, 9, 9]

Evaluation score: 10.00
